In [12]:
import torch
import random

import torch.nn.functional as F
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf
import torch.nn.functional as F
from botorch.acquisition import ExpectedImprovement



In [13]:
BATCHSIZE = 20
TARGET = [0.34, 0.2]


#r = torch.tensor([[random.uniform(0, 1), random.uniform(0, 1)]], dtype=torch.double)  
def dif(tensor, target):
  target = torch.tensor([target], dtype=torch.double)
  r = target - tensor 
  r = torch.abs(r)
  r = torch.sum(r)
  #r = torch.clamp(r,0,1)
  return torch.tensor([[r]], dtype=torch.double)
  


def getSample():
  return torch.tensor([[random.uniform(0, 1), random.uniform(0, 1)]], dtype=torch.double)  

X = torch.tensor([], dtype=torch.double)
Y = torch.tensor([], dtype=torch.double)
# Create a Standardize transform using the data

for i in range(BATCHSIZE):
  x = getSample()
  X = torch.cat((X, x), dim=0)
  y = dif(x, TARGET)
  Y = torch.cat((Y, y), dim=0)
  
  
Y = 1- F.normalize(Y, dim=None, p=2)

print(Y)


list(zip(X, Y))


tensor([[0.0762],
        [0.0642],
        [0.1914],
        [0.2052],
        [0.2917],
        [0.2587],
        [0.0820],
        [0.0726],
        [0.0834],
        [0.0989],
        [0.2126],
        [0.3387],
        [0.0070],
        [0.2071],
        [0.3931],
        [0.4522],
        [0.1660],
        [0.2241],
        [0.1752],
        [0.2455]], dtype=torch.float64)


[(tensor([0.4743, 0.1039], dtype=torch.float64),
  tensor([0.0762], dtype=torch.float64)),
 (tensor([0.5287, 0.1943], dtype=torch.float64),
  tensor([0.0642], dtype=torch.float64)),
 (tensor([0.0635, 0.5028], dtype=torch.float64),
  tensor([0.1914], dtype=torch.float64)),
 (tensor([0.6422, 0.5187], dtype=torch.float64),
  tensor([0.2052], dtype=torch.float64)),
 (tensor([0.9405, 0.4823], dtype=torch.float64),
  tensor([0.2917], dtype=torch.float64)),
 (tensor([0.5377, 0.7853], dtype=torch.float64),
  tensor([0.2587], dtype=torch.float64)),
 (tensor([0.2770, 0.3852], dtype=torch.float64),
  tensor([0.0820], dtype=torch.float64)),
 (tensor([0.2539, 0.3335], dtype=torch.float64),
  tensor([0.0726], dtype=torch.float64)),
 (tensor([0.3793, 0.4131], dtype=torch.float64),
  tensor([0.0834], dtype=torch.float64)),
 (tensor([0.2094, 0.3686], dtype=torch.float64),
  tensor([0.0989], dtype=torch.float64)),
 (tensor([0.8665, 0.3168], dtype=torch.float64),
  tensor([0.2126], dtype=torch.float64)),

In [14]:
train_Y = standardize(Y)
#train_Y = Y
gp = SingleTaskGP(X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)
bounds = torch.stack([torch.zeros(2), torch.ones(2)])

c:\Users\phili\Documents\workspace\botorch\botorch_contrast\.venv\Lib\site-packages\botorch\models\utils\assorted.py:201: InputDataWarning: Input data is not standardized (mean = tensor([0.1923], dtype=torch.float64), std = tensor([0.1171], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


tensor([[1., 1.]])


In [15]:

# Define the Expected Improvement (EI) acquisition function
ei = ExpectedImprovement(gp, best_f=torch.min(Y))

# 'best_observed_value' is the best observed value from your optimization process so far
bounds = torch.stack([torch.zeros(2), torch.ones(2)])

# Optimize the EI acquisition function to find the next candidate point
candidate, acq_value = optimize_acqf(
    ei, bounds=bounds, q=1, num_restarts=20, raw_samples=150,
)

print(candidate)

tensor([[1., 1.]])


In [ ]:

UCB = UpperConfidenceBound(gp, beta=0.1)
candidate, acq_value = optimize_acqf(
    UCB, bounds=bounds, q=1, num_restarts=20, raw_samples=150,
)
print(candidate)

In [ ]:
def optimize_qehvi(model, train_obj, sampler):
    """Optimizes the qEHVI acquisition function, and returns a new candidate and observation."""
    # partition non-dominated space into disjoint rectangles
    partitioning = NondominatedPartitioning(ref_point=ref_point, Y=train_obj)
    acq_func = qExpectedHypervolumeImprovement(
        model=model,
        ref_point=ref_point.tolist(),  # use known reference point
        partitioning=partitioning,
        sampler=sampler,
    )
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=problem_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200, "nonnegative": True},
        sequential=True,
    )
    # observe new values
    new_x =  unnormalize(candidates.detach(), bounds=problem_bounds)
    return new_x